In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy

class InventoryServer(MCPBaseServer):
    def __init__(self):
        super().__init__("inventory-context", "product_inventory")
        self.connector = Connector()
        self.pool = sqlalchemy.create_engine(
            "postgresql+pg8000://",
            creator=self._get_conn,
            pool_pre_ping=True
        )

    def _get_conn(self):
        # IAM-authenticated connection
        return self.connector.connect(
            "my-project:region:instance",
            "pg8000",
            user="agent-service-account@my-project.iam"
        )

    def authenticate(self, request):
        # OAuth 2.1 with inventory_read scope
        token = request.token
        claims = jwt.decode(token, options={"verify_signature": False})
        if not claims.get("scope", "").startswith("inventory_read"):
            raise PermissionError("Invalid scope")

    def fetch_data(self, request):
        # GenAI-generated from query parameters
        query_name = request.json["query"]
        params = request.json["params"]

        predefined_queries = {
            "low_stock": "SELECT * FROM inventory WHERE stock < :threshold",
            "by_category": "SELECT * FROM inventory WHERE category = :cat"
        }

        with self.pool.connect() as conn:
            result = conn.execute(
                sqlalchemy.text(predefined_queries[query_name]),
                params
            ).fetchall()
            return [dict(row) for row in result]

    def build_context(self, raw_data):
        # JSON-LD conversion
        return {
            "@context": "https://schema.org/",
            "@type": "ItemList",
            "itemListElement": [
                {
                    "@type": "Product",
                    "sku": item["product_id"],
                    "name": item["product_name"],
                    "inventory": item["stock"]
                } for item in raw_data
            ],
            "metadata": {
                "source": "CloudSQL",
                "freshness": datetime.datetime.utcnow().isoformat(),
                "row_count": len(raw_data)
            }
        }